In [ ]:
#| default_exp experiments.darts.results

In [ ]:
#| export

from typing import List

import logging
import sys
import pandas as pd
import plotly.graph_objects as go

from national.util.constants import PERCENTAGE_CONFIDENCE_INTERVAL

from national.experiments.darts import predict, evaluate
from national.util import constants

ModuleNotFoundError: No module named 'pandas'

In [ ]:
#| export

logging.disable(sys.maxsize)

In [ ]:
#| export
def _confidence_interval(
    pred: pd.Series,
    stderr: pd.Series,
    percentage_interval,
):

    k = PERCENTAGE_CONFIDENCE_INTERVAL[percentage_interval]

    upper_bound = pred + k * stderr
    lower_bound = pred - k * stderr

    return upper_bound, lower_bound

In [ ]:
#| export
def _metrics(
    models: List[model.Model],
    is_train: bool = True,
):

    _index = pd.Index([x.name for x in models], name='Model')

    if is_train:
        _evaluations = [model.evaluation.train for model in models]
    else:

        _evaluations = [model.evaluation.val for model in models]

    df = pd.DataFrame(
        {
            'sMAPE': [x.smape() for x in _evaluations],
            'rRSME': [x.rrmse() for x in _evaluations],
            'RMSPE': [x.rmspe() for x in _evaluations],
        },
        index=_index,
    )

    df['Dataset'] = 'Train' if is_train else 'Validation'
    return df.reset_index().set_index(['Dataset', 'Model'])

In [ ]:
#| export


class Results(Predict):

    def __init__(
        self,
        **args,
    ):
        super().__init__(**args)

    def plot_model(
        self,
        model: model.Model,
    ):

        return _plot(
            data=self.data,
            model=model,
            freq=self.freq,
            kpi=self.kpi,
            start_at=2,
        )

    def metrics(
        self,
        is_train: bool = True,
    ):
        models = [
            self.models.arima,
            self.models.prophet,
            self.models.mses,
            self.models.partial_ensemble,
        ]
        return _metrics(
            models=models,
            is_train=is_train,
        )

In [ ]:
##%% export

from national.experiments.merlion.results import Results as MerlionResults
results = Results(
    kpi='Sales',
    freq='1D',
    threshold=1,
)

Initial log joint probability = -9.78212
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       1703.58    0.00027321       85.8171      0.1968           1      130   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     110        1703.6   6.18914e-05       99.2942   7.407e-07       0.001      180  LS failed, Hessian reset 
     192        1703.6   1.80674e-07       66.0998           1           1      289   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance
Initial log joint probability = -9.78212
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       1703.58    0.00027321       85.8171      0.1968           1      130   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     110        1703.6   6.18914e-05       99.2942   7.407e-07       0.0

Validation: 100%|██████████| 16243200/16243200 [00:00<00:00, 23188521.19it/s]


Initial log joint probability = -9.78212
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       1703.58    0.00027321       85.8171      0.1968           1      130   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     110        1703.6   6.18914e-05       99.2942   7.407e-07       0.001      180  LS failed, Hessian reset 
     192        1703.6   1.80674e-07       66.0998           1           1      289   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


In [ ]:
##%% export
results.metrics()

sMAPE     rRSME     RMSPE
Dataset Model                                            
Train   Arima               10.304887  0.156351  0.152740
        Prophet             14.806461  0.195130  0.195083
        MSES                34.935393  0.433716  0.424960
        ForecasterEnsemble  10.921829  0.156087  0.152058

In [ ]:
##%% export
results.metrics(is_train=False)

sMAPE     rRSME     RMSPE
Dataset    Model                                            
Validation Arima               17.893858  0.164244  0.164244
           Prophet             22.224818  0.200021  0.200021
           MSES                10.537515  0.100101  0.100101
           ForecasterEnsemble  20.038029  0.182132  0.182132

In [ ]:
##%% export

results.plot_model(results.models.arima)

In [ ]:
##%% export

results.plot_model(results.models.prophet)

In [ ]:
##%% export


results.plot_model(results.models.partial_ensemble)

In [ ]:
##%% export


results.plot_model(results.models.mses)

In [ ]:
kaggle_df = results.to_kaggle('/wd/national/data/submission.csv')

In [ ]:
len(results.data.train)

941